In [ ]:
# Ipython 
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
# Sympy
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
# Yams sympy
from welib.yams.yams_sympy       import YAMSRigidBody, YAMSInertialBody, YAMSFlexibleBody
from welib.yams.yams_sympy_model import YAMSModel

# Inverted (rod) pendulum on a chart

 Point pendulum
 
``` 
 Coordinate system:
 
      
^ y     __   pendulum of mass m_p length l
|       \ \   
|        \ \  
|         \ \~|  angle theta positive about z
|   - c -  \_\|
O-- - k - --[ C ] -------> x 
            chart of mass m_c
           

 ```
 

In [ ]:
mp, mc, k, c, kp, cp, l = sp.symbols('m_p, m_c, k, c, k_p, c_p, l')
gravity = sp.symbols('g')
J_G = sp.symbols('J_G')
J_O = sp.symbols('J_O')
time = dynamicsymbols._t
x, theta = dynamicsymbols('x,theta')
xd, thetad = dynamicsymbols('xd,thetad')
Fx = dynamicsymbols('F_x')

In [ ]:
# --- Bodies, rigid body with COG displaced by d
ref = YAMSInertialBody('E') 
char = YAMSRigidBody('C', mass=mc, rho_G=(0,0,0))
pend = YAMSRigidBody('P', mass=mp, rho_G=(0,l/2,0), J_G =(0,0,J_G)) # NOTE: Inertia defined at COG by default. 
bodies=[char, pend] 
# --- Connections
ref.connectTo(char, type='Free', rel_pos=(x, 0, 0))
char.connectTo(pend, type='Joint', rel_pos=(0, 0, 0), rot_type='Body', rot_amounts=(0,0,theta), rot_order='XYZ')
# --- DOFs and kinetic equations
coordinates = [x,theta]
speeds      = [xd,thetad]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, sp.diff(q, time))]
# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, bdy.mass * gravity * ref.frame.y))] # Watch out for convention here
body_loads += [(char, (char.masscenter,  - k * x * char.frame.x - c * sp.diff(x,time) * char.frame.x))] # stiffness/damping on chard
body_loads += [(char, (char.masscenter,  Fx * char.frame.x))] # control/exteral force
body_loads += [(pend, (pend.frame,  -kp * theta * ref.frame.z - cp * sp.diff(theta,time) * ref.frame.z))] # stiffness/damping on pendulum


# --- Model and equations of motions
model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
EOM=model.to_EOM()
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F